### Imports

In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger
from datetime import datetime
import extras.ourUtils
import numpy as np
import Models

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


### Init

In [2]:
batch_size = 40
full_train = True
path = '/home/jaskmo/Documents/programering/02456DomainAdaptation/'
source_data = path + 'taperImages/pysNetData'
target_data = path + 'taperImages/hData'

## Get data as generators

In [3]:
datagen = ImageDataGenerator(rescale=1./255)

# make a joint generator form a Physionet generator and a Hospital generator
def gen_DAnet(source, target, LL=batch_size, train=True):
    domain_tmp = np.ones(LL, dtype='int8')
    domain_tmp[int(LL/2):] = domain_tmp[int(LL/2):] * 0
    domain_lable = np.concatenate((domain_tmp.reshape(LL,1),np.flip(domain_tmp,0).reshape(LL,1)),1)
    while True:
        if train:
            source_data, source_lable = source.next()
            target_data, target_lable = target.next()
            DA_data = np.concatenate((source_data,target_data),axis=0)
            class_lable = source_lable
            
            yield(DA_data, {'lplOut':class_lable,'dplOut':domain_lable})
        else:
            source_data, source_lable = source.next()
            target_data, target_lable = target.next()
            DA_data = np.concatenate((source_data,target_data),axis=0)
            class_lable = np.concatenate((source_lable,target_lable),axis=0)
        
            yield(DA_data, {'lplOut':class_lable,'dplOut':domain_lable})

#### Train data

In [4]:

train_gen_source = datagen.flow_from_directory(source_data + '/train', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
train_gen_target = datagen.flow_from_directory(target_data + '/train', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
train_gen_DAnet = gen_DAnet(source=train_gen_source, target=train_gen_target, LL=batch_size)

train_stepE = np.floor_divide(train_gen_source.n, batch_size)

Found 29772 images belonging to 5 classes.
Found 10629 images belonging to 5 classes.


#### validation data

In [5]:
valid_gen_source = datagen.flow_from_directory(source_data + '/validation', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
valid_gen_target = datagen.flow_from_directory(target_data + '/validation', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
valid_gen_DAnet = gen_DAnet(source=valid_gen_source, target=valid_gen_target, LL=batch_size)

val_stepE = np.floor_divide(valid_gen_source.n, batch_size)

Found 4807 images belonging to 5 classes.
Found 2838 images belonging to 5 classes.


#### test data

In [6]:
test_gen_source = datagen.flow_from_directory(source_data + '/test', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
test_gen_target = datagen.flow_from_directory(target_data + '/test', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
test_gen_DAnet = gen_DAnet(source=test_gen_source, target=test_gen_target, LL=batch_size)

test_stepE = np.floor_divide(test_gen_source.n, batch_size)

Found 3862 images belonging to 5 classes.
Found 2722 images belonging to 5 classes.


### get model

In [7]:
current_model = Models.DA_model(batch_size=batch_size)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (40, 224, 224, 3)     0                                            
____________________________________________________________________________________________________
vgg16 (Model)                    multiple              14714688    image_input[0][0]                
____________________________________________________________________________________________________
preDa (Flatten)                  (40, 25088)           0           vgg16[1][0]                      
____________________________________________________________________________________________________
lplSplit (Lambda)                (None, 25088)         0           preDa[0][0]                      
___________________________________________________________________________________________

### Callbacks

In [8]:
now = datetime.now()
csv_logger = CSVLogger('/media/jaskmo/ELEK/bme/Project02456/trainingLog/DA_Model' + 
                           str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
                           str(now.hour) + str(now.minute) + '.log')

### Fit the S!@¤

In [9]:
current_model.fit_generator(test_gen_DAnet, steps_per_epoch=train_stepE, validation_data=valid_gen_DAnet, 
                            validation_steps=val_stepE, epochs=10, verbose=1, callbacks=[csv_logger], 
                            )#class_weight=weights_dic)

Epoch 1/10


ValueError: All target arrays (y) should have the same number of samples. Got array shapes: [(20, 5), (40, 2)]